### Notebook to create BLOCK-T188 in Rotator angle with closed loop

Created on: 2024-10-09

Author: Guillem Megias

In [1]:
from lsst.ts.observing import ObservingBlock, ObservingScript 
from lsst.ts.aos.analysis import build_configuration_schema
import os

In [2]:
current_path = os.getcwd()
block_number = 'T188'
program = "BLOCK-T188"
reason = "SITCOM-1491"
note = "LUT_rotator_angle_closed_loop"
constraints = []

### Define configuration schema

In [3]:
# Define the configurable properties that we will use in the configuration schema
properties = {
    "run_filter": {
        "description": "ComCam filter to use.",
        "type": "string",
        "default": "r_03"
    },
    "run_azimuth": {
        "description": "Azimuth to use for the rotator angle sweep.",
        "type": "number",
        "default": 20
    },
    "run_elevation": {
        "description": "Elevation to use for the rotator angle sweep.",
        "type": "number",
        "default": 80
    },
    "run_dofs": {
        "description": "Degrees of freedom to use for the elevation sweep.",
        "type": "array",
        "items": {
            "type": "integer",
            "minimum": 0,
            "maximum": 49,
        },
        "default": list(range(0, 50))
    },
    "run_maxiter": {
        "description": "Maximum number of iterations for the elevation sweep.",
        "type": "integer",
        "default": 5
    }
}

# Build the configuration schema for BLOCK-404
configuration_schema = build_configuration_schema(block_number, properties)
print(configuration_schema)

$schema: http://json-schema.org/draft-07/schema#
title: BLOCK-T188 configuration
description: Configuration for BLOCK-T188.
type: object
properties:
  run_filter:
    description: ComCam filter to use.
    type: string
    default: "r_03"
  run_azimuth:
    description: Azimuth to use for the rotator angle sweep.
    type: number
    default: 20
  run_elevation:
    description: Elevation to use for the rotator angle sweep.
    type: number
    default: 80
  run_dofs:
    description: Degrees of freedom to use for the elevation sweep.
    type: array
    default: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
    items:
      type: integer
      minimum: 0
      maximum: 49
  run_maxiter:
    description: Maximum number of iterations for the elevation sweep.
    type: integer
    default: 5



### Define scripts and block

In [4]:
comcam_closedloop_script = ObservingScript(
    name="maintel/close_loop_comcam.py",
    standard=True,
    parameters= dict(
        filter="$run_filter",
        program="$program",
        reason=reason,
        note=note,
        exposure_time=30.0,
        max_iter="$run_maxiter",
        used_dofs="$run_dofs",
    )
)

stop_tracking_script = ObservingScript(
    name="maintel/stop_tracking.py",
    standard=True,
    parameters = dict()
)

In [5]:
rotator_positions = [-75.0, -63.75, -42.5, -21.25, 0.0, 21.25, 42.5, 63.75, 75.0]
scripts = []

for rotator_angle in rotator_positions:
    track_target_script = ObservingScript(
        name="maintel/track_target.py",
        standard=True,
        parameters = dict(
            track_azel = dict(
                az = '$run_azimuth',
                el = '$run_elevation',
            ),
            rot_value = rotator_angle,
            rot_type = "PhysicalSky",
        )
    )

    scripts.append(track_target_script)
    scripts.append(comcam_closedloop_script)
scripts.append(stop_tracking_script)

In [6]:
block = ObservingBlock(
    name = program,
    program = program,
    configuration_schema=configuration_schema,
    scripts = scripts,
)

### Save configurable block

In [7]:
block.model_dump_json(indent=2)

output_file_path = f'{current_path}/aos/ts_config_ocs/Scheduler/observing_blocks_maintel/AOS/LUTs/{program}.json'

with open(output_file_path, 'w') as file:
    file.write(block.model_dump_json(indent=2))